This program cleans the data on the prediction of well functionality.

In [60]:
import pandas as pd
import numpy as np
import os
import pdb
from tqdm import tqdm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import collections
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:
#os.chdir("C:\\Users\\Xiaotong\\Desktop\\wells")

In [62]:
def load():
    df1 = pd.read_csv("training.csv", parse_dates=["date_recorded"])
    df2 = pd.read_csv("labels.csv")
    df3 = pd.read_csv("test.csv", parse_dates=["date_recorded"])
    df =pd.merge(df2, df1, on="id", how="left")
    return df3, df

In [9]:
def get_composite_names(df, col):
    # list of full name entities
    fullnames_dic = {x: '' for x in df[col]}  # if len(x.split())>=2}
    fullnames_list = [x for x in df[col]]  # if len(x.split())>=2]
    for key in fullnames_dic:
        for i in range(len(key.split())):
            fullnames_dic[key] += key.split()[i][0]
    counter = collections.Counter(fullnames_list)
    return fullnames_dic, fullnames_list, counter

In [11]:
def fuzzymatch_one(counter, tol_partial, tol_sort, tol_set):
    """ match composite names duplicate, starting with the most frequent ones"""
    sorted_keys_desc = [x[0] for x in counter.most_common()]
    sorted_keys_asc = [x[0] for x in sorted(counter.items(), key=lambda x: x[1])]
    matched = {}
    fuzzdic = {}
    for key in tqdm(sorted_keys_desc):
        if (key not in matched) and (key not in fuzzdic):
            matched[key] = [(key, counter[key])]
            fuzzdic[key] = (key, key, counter[key])
            for item in sorted_keys_asc:
                if (item != key) and (item not in fuzzdic):
                    p0 = fuzz.ratio(key, item)
                    p1 = fuzz.partial_ratio(key, item)
                    p2 = fuzz.token_sort_ratio(key, item)
                    p3 = fuzz.token_set_ratio(key, item)
                    if (p1>tol_partial and p3>tol_set) or (p2>tol_sort and p3>tol_set):
                        matched[key].append((item, counter[item]))
                        fuzzdic[item] = (key, item, counter[item])
    return matched, fuzzdic

In [12]:
def hand_match_method1(matched, fuzzdic, key_list, value):
    for key in key_list:
        fuzzdic[key] = (value, key, fuzzdic[key][2]) 
        matched[value].append((key, fuzzdic[key][2]))
        if key in matched:
            for key2 in matched[key]:
                fuzzdic[key2[0]] = (value, key2[0], fuzzdic[key2[0]][2])
            del matched[key]
    return matched, fuzzdic

def hand_match_method2(matched, fuzzdic, value, string):
    for key in fuzzdic:
        if (string in fuzzdic[key][0]) and (key != value):
            fuzzdic[key] = (value, key, fuzzdic[key][2])
            matched[value].append((key, fuzzdic[key][2]))
            if key in matched:
                del matched[key]
    return matched, fuzzdic

def hand_remove(matched, fuzzdic, oldkey, newkey, value):
    fuzzdic[value[0]] = (newkey, value[0], fuzzdic[value[0]][2])
    matched[oldkey].remove(value)
    if newkey in matched:
        matched[newkey].append((value[0], value[1]))
    else:
        matched[newkey] = [(value[0], value[1])]
    return matched, fuzzdic

In [70]:
def shrink_list(fuzzdic, matched, tol):
    for key in matched:
        sum_ = sum([matched[key][i][1] for i in range(len(matched[key]))])
        if sum_ < tol:
            for item in [x for x in matched[key]]:
                fuzzdic[item[0]] = ("local community", item[0], item[1])
    return fuzzdic

In [65]:
def fill_nan(df, col):
    df[col] = df[col].fillna("0")
    df[col] = df[col].str.lower()
    df[col] = df[col].replace(["not know", "not kno"], "0")
    return df

In [63]:
# col = "installer"
col = "scheme_name"

In [64]:
train, test = load()

In [66]:
train = fill_nan(train, col)
test = fill_nan(test, col)

In [67]:
train_names_dic, train_names_list, train_names_counter = get_composite_names(train, col)
test_names_dic, test_names_list, test_names_counter = get_composite_names(test, col)

In [68]:
train_match, train_fuzz = fuzzymatch_one(train_names_counter, 50, 50, 90)

100%|██████████| 1733/1733 [01:16<00:00, 22.70it/s] 


In [69]:
train_match

{'0': [('0', 7092)],
 'k': [('k', 176)],
 'none': [('none', 159)],
 'borehole': [('borehole', 158),
  ('tasaf borehole scheme', 1),
  ('us embassy borehole scheme', 1),
  ('koronani borehole', 1),
  ('bore hole', 1),
  ('borehoole', 1),
  ('michee borehole scheme', 2),
  ('sabodo borehole scheme', 2),
  ('jaica borehole scheme', 13),
  ('gen borehole scheme', 15)],
 'danida': [('danida', 104)],
 'chalinze wate': [('chalinze wate', 96)],
 'm': [('m', 90)],
 'bagamoyo wate': [('bagamoyo wate', 88)],
 'government': [('government', 75)],
 'ngana water supplied scheme': [('ngana water supplied scheme', 65),
  ('kanga water supplied scheme', 18)],
 "wanging'ombe water supply s": [("wanging'ombe water supply s", 62),
  ('s', 34)],
 "uroki-bomang'ombe water sup": [("uroki-bomang'ombe water sup", 57),
  ("uroki bomang'ombe water sup", 2)],
 'n': [('n', 54)],
 'i': [('i', 52)],
 'handeni trunk main(h': [('handeni trunk main(h', 52),
  ('h', 1),
  ('handeni trunk main (', 3)],
 "wanging'ombe supp

In [71]:
train_fuzz = shrink_list(train_fuzz, train_match, 20)

In [72]:
train_fuzz

{'0': ('0', '0', 7092),
 'k': ('k', 'k', 176),
 'none': ('none', 'none', 159),
 'borehole': ('borehole', 'borehole', 158),
 'tasaf borehole scheme': ('borehole', 'tasaf borehole scheme', 1),
 'us embassy borehole scheme': ('borehole', 'us embassy borehole scheme', 1),
 'koronani borehole': ('borehole', 'koronani borehole', 1),
 'bore hole': ('borehole', 'bore hole', 1),
 'borehoole': ('borehole', 'borehoole', 1),
 'michee borehole scheme': ('borehole', 'michee borehole scheme', 2),
 'sabodo borehole scheme': ('borehole', 'sabodo borehole scheme', 2),
 'jaica borehole scheme': ('borehole', 'jaica borehole scheme', 13),
 'gen borehole scheme': ('borehole', 'gen borehole scheme', 15),
 'danida': ('danida', 'danida', 104),
 'chalinze wate': ('chalinze wate', 'chalinze wate', 96),
 'm': ('m', 'm', 90),
 'bagamoyo wate': ('bagamoyo wate', 'bagamoyo wate', 88),
 'government': ('government', 'government', 75),
 'ngana water supplied scheme': ('ngana water supplied scheme',
  'ngana water suppl

In [73]:
train[col] = [train_fuzz[x][0] for x in train[col]]

In [74]:
pd.unique(train[col])

array(['0', 'local community', 'borehole', 'mws',
       'lyamungo umbwe water supply', 'world bank', 'm',
       "wanging'ombe supply scheme", "wanging'ombe water supply s", 'k',
       'machumba estate pipe line', 'machame water supply', 'kijiji',
       'maambreni gravity water supply',
       'nchulowaibale water supply scheme', 'nabaiye pipe line',
       'chalinze wate', "uroki-bomang'ombe water sup", 'upper ruvu',
       'danida', 'i', 'none', 'bagamoyo wate',
       'nasula gravity water supply', 'government',
       'kirua kahe gravity water supply trust', 'n', 'olkokola pipe line',
       'ki', 'handeni trunk main(h', 'kaisho/isingiro w',
       'mtwango water supply scheme', 'tove',
       'ngamanga water supplied sch', 'losaa-kia water supply', 'roman',
       'd', 'sinyanga water supplied sch', 'makwale water supplied sche',
       'mkongoro two', 'u', 'shallow well', 'ngana water supplied scheme',
       'otaruni water supply', 'kirua kahe pumping water trust',
       'ki

In [7]:
train = fill_nan(train, col)
test = fill_nan(test, col)

In [8]:
train["installer"].head()

0          dmdd
1           dwe
2             0
3    finn water
4        bruder
Name: installer, dtype: object

In [10]:
train_names_dic, train_names_list, train_names_counter = get_composite_names(train, col)
test_names_dic, test_names_list, test_names_counter = get_composite_names(test, col)

In [13]:
train_match, train_fuzz = fuzzymatch_one(train_names_counter, 50, 50, 90)

100%|██████████| 978/978 [00:15<00:00, 62.67it/s] 


In [14]:
key_list = ['rc ch', 'rc c', 'roman church', 'roman catholic', 'roman cathoric -kilomeni',
            'roman cathoric -same', 'rc cathoric', 'rulenge diocese', 'roman catholic rulenge diocese',
           'roman ca']
value = 'rc church'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [15]:
key_list = ['central govt', 'tanzania government', 'tcrs /government', 'concern /government', 'adra /government',
            'ministry of water', 'ministry of healthy', 'central government']
value = 'government'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [16]:
key_list = ['consultant engineer', 'citizen engine', 'howard and humfrey consultant']
value = 'consulting engineer'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [17]:
key_list = ['local te', 'local contract', 'local fundi', 'local technical tec', 'local technical']
value = 'local  technician'
train_match, train_fuzz = hand_match_method1(train_match, train_fuzz, key_list, value)

In [18]:
string = 'kkt'
value = 'kkkt'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [19]:
train_match, train_fuzz = hand_remove(train_match, train_fuzz, "government", "local government", ("local government", 1))

In [20]:
string = 'local t'
value = 'kkkt church'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [21]:
string = 'italy'
value = 'italian government'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [22]:
string = 'would bank'
value = 'world bank'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

string = 'would vission'
value = 'world vision'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

string = 'village'
value = 'village council'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

string = 'china henan contractor'
value = 'china henan construction'
train_match, train_fuzz = hand_match_method2(train_match, train_fuzz, value, string)

In [23]:
check = {key: value for key, value in train_match.items() if type(value)!=str}
[x for x in check.values()]

[[('dwe', 4351),
  ('ubalozi wa marekani/dwe', 1),
  ('consultant and dwe', 1),
  ('lgsp/dwe', 1),
  ('dwe/', 1),
  ('water aid/dwe', 3),
  ('dwe}', 7),
  ('rwe/dwe', 12),
  ('kkkt _ konde and dwe', 43)],
 [('0', 1081)],
 [('government', 476),
  ('colonial government', 1),
  ('british colonial government', 1),
  ('cebtral government', 1),
  ('village government', 1),
  ('italy government', 1),
  ('tcrs /government', 1),
  ('japan government', 1),
  ('adra /government', 1),
  ('central government/tlc', 1),
  ('belgiam government', 2),
  ('concern /government', 2),
  ('italian government', 2),
  ('isf/government', 4),
  ('tanzania government', 4),
  ('cipro/government', 5),
  ('finland government', 7),
  ('central government', 170),
  ('central govt', 37),
  ('tanzania government', 4),
  ('tcrs /government', 1),
  ('concern /government', 2),
  ('adra /government', 1),
  ('ministry of water', 6),
  ('ministry of healthy', 1),
  ('central government', 170)],
 [('hesawa', 373), ('heasawa', 

In [24]:
train_fuzz

{'0': ('0', '0', 1081),
 'aar': ('aar', 'aar', 1),
 'abasia': ('abasia', 'abasia', 5),
 'abdi mtili': ('abdi mtili', 'abdi mtili', 1),
 'abdul': ('abdul', 'abdul', 1),
 'abraham palanjo': ('abraham palanjo', 'abraham palanjo', 1),
 'ac': ('ac', 'ac', 3),
 'accra': ('accra', 'accra', 9),
 'acord': ('acord', 'acord', 1),
 'acra': ('acra', 'acra', 68),
 'act': ('act', 'act', 1),
 'action aid': ('action aid', 'action aid', 3),
 'action contre la faim': ('action contre la faim',
  'action contre la faim',
  3),
 'active kmk': ('active kmk', 'active kmk', 1),
 'active mkm': ('active mkm', 'active mkm', 7),
 'active tank co': ('co', 'active tank co', 2),
 'ad': ('ad', 'ad', 3),
 'adap': ('adap', 'adap', 1),
 'adb': ('adb', 'adb', 1),
 'adp': ('adp', 'adp', 4),
 'adp busangi': ('adp', 'adp busangi', 4),
 'adra': ('adra', 'adra', 42),
 'adra /community': ('community', 'adra /community', 7),
 'adra /government': ('government', 'adra /government', 1),
 'adra/ community': ('community', 'adra/ comm

In [25]:
# def popular_acronyms(matched, df, col, tol):
#     allnames_list = [x for x in df[col]]
#     counter = collections.Counter(allnames_list)
#     acro = {}
#     for key in matched:
#         sum_ = sum([matched[key][i][1] for i in range(len(matched[key]))])
#         if  sum_ > tol:
#             acronym = [key.split()[i][0] for i in range(len(key.split()))]
#             acro[''.join(acronym)] = [(key, sum_)]
#     return acro

In [26]:
# acronyms = popular_acronyms(train_match, train, col, 10)

In [27]:
# acronyms

In [28]:
# def match_acronyms(fuzzdic, df, acronyms):
#     allnames_list = [x for x in df[col]]
#     counter = collections.Counter(allnames_list)
#     for key in counter:
#         if (key not in fuzzdic) and (key in acronyms):
#             acronyms[key].append((key, counter[key]))
#     return acronyms

In [29]:
# acronyms = match_acronyms(train_fuzz, train, acronyms)

In [30]:
# acronyms

In [31]:
# train_histo = np.histogram(list(trainnames_counter.values()), bins=len(trainnames_counter.values()))
# test_histo = np.histogram(list(testnames_counter.values()), bins=len(testnames_counter.values()))

In [32]:
# plt.bar(train_histo[1][:-1], train_histo[0])
# plt.ylim((0,30))
# plt.xlim(0,100)
# plt.xlabel("number of occurences of a name")
# plt.ylabel("number of names that occur x times")

In [33]:
# plt.bar(test_histo[1][:-1], test_histo[0])
# plt.ylim((0,30))
# plt.xlim(0,100)
# plt.xlabel("number of occurences of a name")
# plt.ylabel("number of names that occur x times")